In [ ]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import A2C
from stable_baselines.common.env_checker import check_env
from IPython.display import clear_output
import time
from stable_baselines.common.evaluation import evaluate_policy
from copy import deepcopy
from ray import tune
from stable_baselines.common.callbacks import BaseCallback

In [ ]:
import gym
import w_mac
from collections import defaultdict
import matplotlib as plt
import networkx as nx

# data = [(0,2),(0,1),(1,2),(2,3),(2,4),(3,4)]
#data = [(0,1),(0,8),(0,7),(0,12),(0,2),(1,2),(1,12),(1,5),(1,6),(1,14),(1,7),(1,8),(5,6),(5,14),(6,14),(8,7),(12,2),(2,3),(2,4),(4,13),(4,9),(4,11),(4,10),(10,11),(9,13),(3,4),(9,10),(9,11),(5,15)]
d = defaultdict(list)
data = [(0,2),(0,1),(0,3),(1,2),(1,3),(2,3),(2,4),(3,4),(5,2),(5,3),(5,4)]
# defaultdict(<type 'list'>, {})
for node, dest in data:
    d[node].append(dest)
print(d)

G = nx.Graph()
for k,v in d.items():
    for vv in v:
        G.add_edge(k,vv)
nx.draw(G)

In [ ]:
env = gym.make('wmac-graph-v0',graph=G)
env.reset()

In [ ]:
check_env(env)

In [ ]:
import tensorflow as tf

class TensorboardCallback(BaseCallback):
    """
    Custom callback for plotting additional values in tensorboard.
    """
    def __init__(self, verbose=0):
        self.is_tb_set = False
        super(TensorboardCallback, self).__init__(verbose)

    def _on_step(self) -> bool:
        # Log additional tensor
        if not self.is_tb_set:
            with self.model.graph.as_default():
                tf.summary.scalar('packet_lost', tf.reduce_mean(env.get_packet_lost()))
                self.model.summary = tf.compat.v1.summary.merge_all()
            self.is_tb_set = True
        # Log scalar value (here a random variable)
        value = env.get_packet_lost()
        summary = tf.Summary(value=[tf.Summary.Value(tag='packet_lost', simple_value=value)])
        self.locals['writer'].add_summary(summary, self.num_timesteps)
        return True

In [ ]:
model = A2C(MlpPolicy, env, verbose=1,tensorboard_log="./a2c_tensorboard/")

model.learn(total_timesteps=400000, callback=TensorboardCallback())
model.save("a2c_wmac")

In [ ]:
del model # remove to demonstrate saving and loading

In [ ]:
model = A2C.load("a2c_wmac")

obs = env.reset()
count = 0
while count < 100:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    #env.render()
    count = count + 1
    #print("count :", count)
    if done:
        # clear_output(wait = True)
        time.sleep(3)
        #env.render()
        break


In [ ]:
# def evaluate_objective(config):
#     tune_env = deepcopy(env)
#     tune_agent = A2C("MlpPolicy", tune_env, learning_rate= config["lr"] )#**config)
#     tune_agent.learn(total_timesteps=10)
    
#     mean_reward, std_reward = evaluate_policy(tune_agent, env, n_eval_episodes=10, render=False,
#                                                            deterministic=True,
#                                                            return_episode_rewards=False)
#     print("mr",mean_reward,"sd", std_reward)
#     tune.report(mean_reward=mean_reward)
    
    

In [ ]:
# from ray import tune

# analysis = tune.run(
#     evaluate_objective,
#     #num_samples=3,
#     config={
#         "lr": tune.grid_search([0.001, 0.01, 0.1]),
#     },
#     metric="mean_reward",
#     mode="max"
#     )

In [ ]:
# best_config = analysis.best_config
# print(best_config)